In [2]:
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Unit import *


## PySpice 101

PySpice is a circuit simulator with a text interface. We are accustomed to viewing an electrical ciruit as a circuit diagram. However, representing a circuit to a computer for analysis requires a different approach. PySpice has a text interface for the purpose.

Think of the circuit diagram as a network graph. Such a graph, in Computer Science, has nodes and edges. In a circuit, junctions are nodes and components (capacitor, transistor, op amp, etc.) are edges. This is an elegant representation becuase circuit analysis examines voltage or current at a junction (with respect to ground). This manner of representation of a circuit is called *netlist* in PySpice. 

Here is an example [multisource DC circuit](https://www.allaboutcircuits.com/uploads/articles/multiple-source-DC-resistor-network-circuit-example-1.jpg): 

<img src="https://www.allaboutcircuits.com/uploads/articles/multiple-source-DC-resistor-network-circuit-example-1.jpg">

``` 
Multiple dc sources v1 1 0 dc 24 v2 3 0 dc 15 r1 1 2 10k r2 2 3 8.1k r3 2 0 4.7k .end 
```

This compact representation has the followiing components:
- The label "Multiple dc sources" 
- DC voltage source #1 with name 'v1', spanning nodes 0 (GND) and 1, type dc and size 24 volts
- DC voltage source #2 with name 'v2', spanning nodes 0 (GND) and 3, type dc and size 15 volts
- Resistor named 'r1' spanning nodes 1 and 2 and size 10k ohms
- Resistor named 'r2' spanning nodes 2 and 3 and size 8.1k ohms
- Resistor named 'r3' spanning nodes 0 (GND) and size 2 and size 4.7k ohms.
It has only the essential information about the circuit and no fluff.

The netlist also includes provision to specify the type of simulation and content to report in output. We may want to examine the voltage or current in any of the following ways:
1. steady-state
2. transient
3. time-varying

PySpice offers these possibilities as relevant for AC or DC circuit analysis. See [examples](https://www.allaboutcircuits.com/textbook/reference/chpt-7/example-circuits-and-netlists/).

Let's start with a test circuit with 2 resistors in parallel.

## Resistors in Parallel

In [41]:
class ParallelResistors(SubCircuit):
    __nodes__ = ('01', '02')
    def __init__(self, name, R1=1@u_Ω, R2=2@u_Ω):
        SubCircuit.__init__(self, name, *self.__nodes__)
        self.R("Resistor_One", '01', '02', R1)
        self.R("Resistor_Two", '01', '02', R2)

In [42]:
circuit = Circuit('Test')
circuit.subcircuit(ParallelResistors('Part_One', R2=2@u_Ω))
circuit.X('1', 'Part_One', 1, circuit.gnd)
circuit.subcircuit(ParallelResistors('Part_Two', R2=3@u_Ω))
circuit.X('2', 'Part_Two', 1, circuit.gnd)
print(circuit)

.title Test
.subckt Part_One 01 02
RResistor_One 01 02 1Ohm
RResistor_Two 01 02 2Ohm
.ends Part_One

.subckt Part_Two 01 02
RResistor_One 01 02 1Ohm
RResistor_Two 01 02 3Ohm
.ends Part_Two
X1 1 0 Part_One
X2 1 0 Part_Two

